In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# ls

In [10]:
# cd drive/MyDrive

In [11]:
import os
import cv2
from PIL import Image
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from tensorflow.keras.utils import img_to_array
# from tensorflow.keras.optimizers import Adam, RMSprop
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
imagegen = ImageDataGenerator()
import matplotlib.pyplot as plt

In [12]:
train = imagegen.flow_from_directory("Marine_Dataset/train/", class_mode="categorical", shuffle=True, batch_size=32, target_size=(50, 50))
test = imagegen.flow_from_directory("Marine_Dataset/test/", class_mode="categorical", shuffle=True, batch_size=32, target_size=(50, 50))
val = imagegen.flow_from_directory("Marine_Dataset/val/", class_mode="categorical", shuffle=True, batch_size=32, target_size=(50, 50))

Found 4642 images belonging to 4 classes.
Found 1037 images belonging to 4 classes.
Found 115 images belonging to 4 classes.


In [13]:
pre_trained_model = tf.keras.applications.VGG19(input_shape=(50, 50, 3), include_top=False, weights="imagenet")

In [14]:
for layer in pre_trained_model.layers:
  print(layer.name)
  layer.trainable = False

input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_conv4
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_conv4
block5_pool


In [15]:
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
x = tf.keras.layers.GlobalMaxPooling2D()(last_output)
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(4, activation='sigmoid')(x)

In [16]:
model = tf.keras.Model(pre_trained_model.input, x)

In [17]:
model.compile(optimizer='rmsprop', loss=tf.keras.losses.categorical_crossentropy, metrics=['accuracy'], run_eagerly=True)

In [18]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50, 50, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 50, 50, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 50, 50, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 25, 25, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 25, 25, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 25, 25, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

In [19]:
vgg_classifier = model.fit(train, steps_per_epoch=(2520//32), epochs = 5, validation_data=val, validation_steps=(33//32), batch_size = 32, verbose = 1)

Epoch 1/5
78/78 [==============================] - 214s 3s/step - loss: 4.4255 - accuracy: 0.7032 - val_loss: 5.8127 - val_accuracy: 0.7500
Epoch 2/5
78/78 [==============================] - 109s 1s/step - loss: 2.4387 - accuracy: 0.7660 - val_loss: 11.4367 - val_accuracy: 0.6562
Epoch 3/5
78/78 [==============================] - 58s 749ms/step - loss: 1.5393 - accuracy: 0.8049 - val_loss: 1.6885 - val_accuracy: 0.7500
Epoch 4/5
78/78 [==============================] - 31s 395ms/step - loss: 1.1480 - accuracy: 0.8205 - val_loss: 3.0285 - val_accuracy: 0.6875
Epoch 5/5
78/78 [==============================] - 13s 167ms/step - loss: 0.8348 - accuracy: 0.8260 - val_loss: 2.4274 - val_accuracy: 0.8438


In [20]:
result = model.evaluate(test, batch_size=32)
print("[Loss, Accuracy] = ", result)

33/33 [==============================] - 291s 9s/step - loss: 0.8600 - accuracy: 0.8341
[Loss, Accuracy] =  [0.8600058555603027, 0.834136962890625]


In [21]:
import numpy as np

In [22]:
test_img = 'test_image.png'

In [23]:
test = cv2.imread(test_img)
test = cv2.resize(test, (50, 50))
test = np.array(test)
test = np.expand_dims(test, axis=0)
print("Shape of Validation Data: ", test.shape)
pred = model.predict(test)

Shape of Validation Data:  (1, 50, 50, 3)
1/1 [==============================] - 0s 321ms/step


In [24]:
def pred_img(index):
    if index == 0:
        return 'Biodegradable'
    elif index == 1:
        return 'Non-Biodegradable'
    elif index == 3:
        return 'Reduce/Reuse/Recycle-able'
    else:
        return 'Others'


op = []
pred = np.array(pred.tolist())[0]
index = pred.tolist().index(max(pred))
# print(index)
print("Input Test Image falls under the Category:", pred_img(index))
print("Accuracy = ", max(pred))

Input Test Image falls under the Category: Non-Biodegradable
Accuracy =  0.9917866587638855
